#Movie Recommendations System

### Import dan Load data

In [1]:
import pandas as pd

# Membaca data film dari file CSV
movies = pd.read_csv("movies.csv")

# Menampilkan 5 baris pertama data film
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### Preprocessing Data - Membersihkan Judul Film

In [2]:
import re

def clean_title(title):
    # Menghapus karakter non alfanumerik kecuali spasi
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

# Membuat kolom baru 'clean_title' dengan judul yang sudah dibersihkan
movies["clean_title"] = movies["title"].apply(clean_title)

### Vektorisasi Judul Film dengan TF-IDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Membuat objek TF-IDF vectorizer dengan n-gram unigrams dan bigrams
vectorizer = TfidfVectorizer(ngram_range=(1,2))

# Melakukan fit transform pada judul film yang sudah dibersihkan
tfidf = vectorizer.fit_transform(movies["clean_title"])

### Fungsi Pencarian Film Berdasarkan Judul

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)  # Bersihkan input judul
    query_vec = vectorizer.transform([title])  # Vektorkan input
    similarity = cosine_similarity(query_vec, tfidf).flatten()  # Hitung kemiripan cosine
    indices = np.argpartition(similarity, -5)[-5:]  # Ambil indeks 5 film teratas
    results = movies.iloc[indices].iloc[::-1]  # Ambil data film dari indeks, dibalik urutan

    return results

### User Interface untuk Pencarian Film (Menggunakan ipywidgets)

In [5]:
import ipywidgets as widgets
from IPython.display import display

# Widget input teks untuk memasukkan judul film
movie_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)

# Output area untuk menampilkan hasil pencarian
movie_list = widgets.Output()

# Fungsi yang dijalankan setiap kali input teks berubah
def on_type(data):
    with movie_list:
        movie_list.clear_output()  # Bersihkan output sebelumnya
        title = data["new"]
        if len(title) > 5:  # Jika input sudah cukup panjang (lebih dari 5 karakter)
            display(search(title))  # Tampilkan hasil pencarian film

movie_input.observe(on_type, names='value')  # Pasang observer pada input

# Tampilkan widget input dan area hasil pencarian
display(movie_input, movie_list)


Text(value='Toy Story', description='Movie Title:')

Output()

## Load Data Rating dan Analisis Rekomendasi Film Mirip

In [6]:
ratings = pd.read_csv("ratings.csv")

# Tipe data kolom dalam dataset rating
ratings.dtypes

,0
userId,int64
movieId,int64
rating,float64
timestamp,int64


### Fungsi Mencari Film Rekomendasi Berdasarkan User yang Memberi Rating Tinggi Film Tertentu

In [7]:
def find_similar_movies(movie_id):
    # Cari user yang memberikan rating > 4 pada film tertentu
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

    # Ambil rekomendasi film lain yang juga diberi rating > 4 oleh user tersebut
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

    # Hitung frekuensi rekomendasi dan normalisasi dengan jumlah user mirip
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    # Filter rekomendasi yang muncul di lebih dari 10% user mirip
    similar_user_recs = similar_user_recs[similar_user_recs > .10]

    # Ambil data rating dari semua user yang memberikan rating > 4 pada film-film tersebut
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

    # Gabungkan persentase rekomendasi dari user mirip dan semua user
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    # Hitung skor dengan membandingkan frekuensi user mirip dan semua user
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

    # Urutkan berdasarkan skor tertinggi
    rec_percentages = rec_percentages.sort_values("score", ascending=False)

    # Gabungkan dengan data film untuk menampilkan judul dan genre
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]


### User Interface untuk Rekomendasi Berdasarkan Judul Film

In [8]:
movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]  # Ambil movieId film teratas hasil pencarian
            display(find_similar_movies(movie_id))  # Tampilkan rekomendasi film

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()